# Three-point estimation

In [1]:
from IPython.display import HTML


HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
class Task(object):
    def __init__(self, text_w, del_button, best, likely, worst, container):
        self.is_active = True
        
        self.text_w = text_w
        self.del_button = del_button
        self.best = best
        self.likely = likely
        self.worst = worst
        self.container = container
        
        self.estimation = None
        self.deviation = None
        
    def calculate(self):
        a = self.best.value
        m = self.likely.value
        b = self.worst.value
        
        self.estimation = (a + 4 * m + b) / 6
        self.deviation = (b - a) / 6
    
    def deactivate(self):
        self.is_active = False
        self.container.close()


In [3]:
import ipywidgets as widgets


tasks = {}
def add_task_widget(button):
    task_id = max(tasks.keys(), default=0) + 1
    text = widgets.Text(
        value='Task #{}'.format(task_id),
        placeholder='Task name',
    )
    del_button = widgets.Button(
        description='',
        tooltip='Delete task',
        icon='minus',
        button_style='danger',
    )
    best = widgets.FloatText(
        description='Best-case estimate:',
    )
    likely = widgets.FloatText(
        description='Most likely estimate:',
    )
    worst = widgets.FloatText(
        description='Worst-case estimate:',
    )
    
    task_hboxes = (
        widgets.HBox((text, del_button)),
        widgets.VBox((best, likely, worst)),
    )
    task_vbox = widgets.VBox(task_hboxes)
    task = Task(text, del_button, best, likely, worst, container=task_vbox)
    
    del_button.on_click(lambda b: task.deactivate())

    tasks[task_id] = task
    display(task_vbox)
    

button = widgets.Button(
    description='Add task',
    tooltip='Add one more task',
    icon='plus',
)
button.on_click(add_task_widget)
button

Button(description='Add task', icon='plus', style=ButtonStyle(), tooltip='Add one more task')

In [4]:
import math

APPROACHES = (
    (68, 1),
    (90, 1.645),
    (95, 2),
    (99.7, 3),
)
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

def recalculate():
    proj_estimation = 0
    proj_error = 0
    for task in tasks.values():
        if task.is_active:
            task.calculate()
            proj_estimation += task.estimation
            proj_error += task.deviation ** 2
    
    proj_error = math.sqrt(proj_error)
    
    return proj_estimation, proj_error


def show_estimations(b):
    msg = 'The {confidence}% confidence interval for the true project work time is approximately: {estimation} hours ({minmax})'
    proj_estimation, proj_error = recalculate()
    
    out.clear_output()
    
    for approach in APPROACHES:
        confidence, multiplier = approach
        error = proj_error * multiplier
        
        minmax = '{:.1f}h - {:.1f}h'.format(proj_estimation - error, proj_estimation + error)
        estimation = '{:.1f} ± {:.1f}'.format(proj_estimation, error)
        
        with out:
            print(msg.format(confidence = confidence, estimation = estimation, minmax = minmax)) 

button = widgets.Button(
    description='Calculate',
    tooltip='Calculate project estimation',
    button_style='success',
)
button.on_click(show_estimations)
button
    

Output(layout=Layout(border='1px solid black'))

Button(button_style='success', description='Calculate', style=ButtonStyle(), tooltip='Calculate project estima…